In [1]:
from copy import deepcopy
import regex as re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
from importlib import reload
import tweets, users, places, twitter_data
import cleaning as c
import logs
import configs
import files

In [2]:
# Configuration files
gen_conf = configs.read_conf()
conf = configs.read_conf('c')

In [3]:
# Extracted folder to clean
e_path = files.choose_save_path('e')
t_path = files.get_project_root()/'tests'

Choose from the available folders, comma-separated (or "a" for all):
0. 2021-07-26-at-15:46:00
1. 2021-07-26-at-23:45:00
2. 2021-11-07-at-22:10:00
3. 2023-02-08-at-23:05:39
4. 2021-07-26-at-19:05:00
5. 2023-02-22-at-18:46:25
6. 2021-07-07-at-10:50:00
7. 2023-02-22-at-18:13:40
Return folder(s): 2


In [4]:
# Path to the verb conjugations
conjug_es = files.get_verb_conjugations()

In [51]:
sample_path = list(e_path[0].iterdir())[0]
sample_path

PosixPath('/home/rimov/Documents/Code/NLP/lin-que-dropping/data/extracted/saved/twitter/es/2021-11-07-at-22:10:00/es-parecer-original-tweets-10083-0.csv')

In [52]:
sample = tweets.Tweets.from_csv(sample_path, 'es', 'df-sample')
display(sample.d.head())
display(sample.d.info())

,tweet_id,text_orig,author_id,referenced_tweets,lang,created_at,mentions,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,tweet_place_id,text_norm,geo.coordinates.type,geo.coordinates.coordinates
0,1457563208949059587,@lpamx El tipo es un ridículo y discute con un...,332090173,"[{'type': 'replied_to', 'id': '145755916849218...",es,2021-11-08 04:18:43,"[{'start': 0, 'end': 6, 'username': 'lpamx', '...",0,0,1,0,1e6d711c280456ce,El tipo es un ridiculo y discute con un CM. I...,NaN,NaN
1,1457563016602558471,Algo extraño está sucediendo en el país y pare...,177217287,<NA>,es,2021-11-08 04:17:58,<NA>,0,0,1,0,009924a469d7ace1,Algo extrano esta sucediendo en el pais y pare...,NaN,NaN
2,1457562948512194568,@mavillanita 😂😂😂 Lo importante es que te haga...,49068316,"[{'type': 'replied_to', 'id': '145756257799580...",es,2021-11-08 04:17:41,"[{'start': 0, 'end': 12, 'username': 'mavillan...",0,0,0,0,00ec4c32070f5076,Lo importante es que te haga feliz y tal pare...,NaN,NaN
3,1457561866452942851,@pinabertoglia Hay historias asi en el mundo d...,44260132,"[{'type': 'replied_to', 'id': '145747260304866...",es,2021-11-08 04:13:23,"[{'start': 0, 'end': 14, 'username': 'pinabert...",0,0,0,0,010a74892213c898,"Hay historias asi en el mundo del vino, y mejo...",NaN,NaN
4,1457561439888105473,Hasta parece q Lennon sigue gobernando. Torpez...,1512320078,<NA>,es,2021-11-08 04:11:42,<NA>,0,0,1,0,0177bc687a9ac697,Hasta parece q Lennon sigue gobernando. Torpez...,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10083 entries, 0 to 10082
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   tweet_id                      10083 non-null  UInt64        
 1   text_orig                     10083 non-null  string        
 2   author_id                     10083 non-null  UInt64        
 3   referenced_tweets             6856 non-null   string        
 4   lang                          10083 non-null  category      
 5   created_at                    10083 non-null  datetime64[ns]
 6   mentions                      6964 non-null   string        
 7   public_metrics.retweet_count  10083 non-null  UInt32        
 8   public_metrics.reply_count    10083 non-null  UInt32        
 9   public_metrics.like_count     10083 non-null  UInt32        
 10  public_metrics.quote_count    10083 non-null  UInt32        
 11  tweet_place_id              

None

In [53]:
sample.d = sample.d.sample(10, random_state=1000, ignore_index=True)
display(sample.d.head(3))
display(sample.d.info())

,tweet_id,text_orig,author_id,referenced_tweets,lang,created_at,mentions,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,tweet_place_id,text_norm,geo.coordinates.type,geo.coordinates.coordinates
0,1456442983461519360,@jesusibarra0 @CiudadanoAntic1 @cpsseajal @Zap...,790388021233225728,"[{'type': 'replied_to', 'id': '145604905055024...",es,2021-11-05 02:07:21,"[{'start': 0, 'end': 13, 'username': 'jesusiba...",0,0,0,0,8e098f6d686001a9,Parece otro ejercicio para que el perfiles a s...,NaN,NaN
1,1455392705488596994,@paolits_c Parece que quiere voltear la novela...,3025719424,"[{'type': 'replied_to', 'id': '145537692962073...",es,2021-11-02 04:33:55,"[{'start': 0, 'end': 10, 'username': 'paolits_...",0,0,2,0,003aa1d3152e364a,"Parece que quiere voltear la novela, cosa de l...",NaN,NaN
2,1456141150197272580,hasta que le he dejado en evidencia delante de...,721102522090393600,"[{'type': 'replied_to', 'id': '145614114765132...",es,2021-11-04 06:07:58,<NA>,0,0,1,0,d95b50d40bcc459b,hasta que le he dejado en evidencia delante de...,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   tweet_id                      10 non-null     UInt64        
 1   text_orig                     10 non-null     string        
 2   author_id                     10 non-null     UInt64        
 3   referenced_tweets             6 non-null      string        
 4   lang                          10 non-null     category      
 5   created_at                    10 non-null     datetime64[ns]
 6   mentions                      5 non-null      string        
 7   public_metrics.retweet_count  10 non-null     UInt32        
 8   public_metrics.reply_count    10 non-null     UInt32        
 9   public_metrics.like_count     10 non-null     UInt32        
 10  public_metrics.quote_count    10 non-null     UInt32        
 11  tweet_place_id                10 no

None

In [54]:
keep_cols = ['tweet_id', 'text_orig']

In [55]:
sample.d = sample.d.loc[:, keep_cols]
sample.d.head()

,tweet_id,text_orig
0,1456442983461519360,@jesusibarra0 @CiudadanoAntic1 @cpsseajal @Zap...
1,1455392705488596994,@paolits_c Parece que quiere voltear la novela...
2,1456141150197272580,hasta que le he dejado en evidencia delante de...
3,1456272375859740673,@diariodeavisos Qué será lo que está fallando...
4,1457205580544815105,@CalixxxtoMM @AngelMemer0 @JakeMemero07 @capis...


In [56]:
dup_data = sample.d.sample(3, random_state=1000)
tweet_ids = set(str(i) for i in sample.d['tweet_id'].unique())
display(dup_data)
display(tweet_ids)

,tweet_id,text_orig
2,1456141150197272580,hasta que le he dejado en evidencia delante de...
6,1455332378780504069,@Nereida54 Estamos en un punto importante de C...
5,1455304983897788417,A mi siempre me parecerá muy loco que en lo qu...


{'1455304983897788417',
 '1455332378780504069',
 '1455366114343624716',
 '1455392705488596994',
 '1456114160312389645',
 '1456141150197272580',
 '1456272375859740673',
 '1456442983461519360',
 '1457205580544815105',
 '1457482258714136576'}

In [57]:
sample.d = pd.concat([sample.d, dup_data], axis=0, ignore_index=True)
sample.d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   13 non-null     UInt64
 1   text_orig  13 non-null     string
dtypes: UInt64(1), string(1)
memory usage: 349.0 bytes


In [58]:
sample.d = sample.d.rename(columns={'tweet_id': 'id'})

In [59]:
sample.save(t_path/'samples'/'id-keeping', 'csv')

[PosixPath('/home/rimov/Documents/Code/NLP/lin-que-dropping/tests/samples/id-keeping/es-twitter-df-sample-tweets-13.csv')]

## <code>TwitterData.update_ids()</code> samples

In [62]:
p_id_keeping = files.get_project_root()/'tests'/'samples'/'id-keeping'
p_exist_ids = p_id_keeping/'ids-existing.txt'
p_actual_ids = p_id_keeping/'ids-actual.txt'

In [63]:
existing_ids = set(p_exist_ids.read_text().split())
print(existing_ids)

{'1', '4', '3', '10', '5', '1455392705488596994', '15', '2', '25', '1456272375859740673', '20'}


In [64]:
actual_ids = tweet_ids.union(existing_ids)
actual_ids

{'1',
 '10',
 '1455304983897788417',
 '1455332378780504069',
 '1455366114343624716',
 '1455392705488596994',
 '1456114160312389645',
 '1456141150197272580',
 '1456272375859740673',
 '1456442983461519360',
 '1457205580544815105',
 '1457482258714136576',
 '15',
 '2',
 '20',
 '25',
 '3',
 '4',
 '5'}

In [65]:
p_actual_ids.write_text(' '.join([i for i in actual_ids]))

221

## Getting Column Names Across All Extracted Data

In [25]:
e_path = files.choose_save_path('e')

Choose from the available folders, comma-separated (or "a" for all):
0. 2021-07-26-at-15:46:00
1. 2021-07-26-at-23:45:00
2. 2021-11-07-at-22:10:00
3. 2023-02-08-at-23:05:39
4. 2021-07-26-at-19:05:00
5. 2023-02-22-at-18:46:25
6. 2021-07-07-at-10:50:00
7. 2023-02-22-at-18:13:40
Return folder(s): 5


In [38]:
ptweets = e_path[0]/'tweets'
pusers = e_path[0]/'users'
pplaces = e_path[0]/'places'
pmeta = e_path[0]/'twitterdata'

In [ ]:
psample = places.Places.from_csv(pplaces/'es-twitter-decir-places-1-470.csv')

In [42]:
tweets_sample = tweets.Tweets.from_csv(
    [p for p in ptweets.iterdir()][0], 'es'
)
users_sample = users.Users.from_csv(
    [p for p in pusers.iterdir()][0], 'es'
)
places_sample = places.Places.from_csv(
    [p for p in pplaces.iterdir()][0], 'es'
)
meta_sample = twitter_data.TwitterData.from_csv(
    [p for p in pmeta.iterdir()][0], 'es'
)

ValueError: Unable to parse string "bc1dbfe09834a716" at position 0

In [45]:
places_sample.d.head()

NameError: name 'places_sample' is not defined